In [1]:
import os
import json
import pandas as pd
import traceback



In [2]:
from langchain.chat_models import ChatOpenAI

In [6]:
keys=os.getenv("OPENAI_KEY")

In [4]:
llm=ChatOpenAI(openai_api_key=keys.openaikey, model_name="gpt-3.5-turbo",temperature=0.3)

AttributeError: 'str' object has no attribute 'openaikey'